In [10]:
# Data Loader
import os
import csv
from PIL import Image
import pillow_heif  # Import pillow-heif to enable HEIC support in PIL
import numpy as np
from tqdm import tqdm  # For progress tracking

class DataLoader:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
    
    def create_labels_csv(self, output_file='labels.csv'):
        """Generates a CSV file with image filenames, ranks, and suits."""
        with open(output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['filename', 'rank', 'suit'])
            for folder_name in os.listdir(self.dataset_path):
                folder_path = os.path.join(self.dataset_path, folder_name)
                if os.path.isdir(folder_path):
                    rank, suit = folder_name.split('_')
                    for image_name in os.listdir(folder_path):
                        # Only process files with image extensions
                        if image_name.lower().endswith(('.jpg', '.jpeg', '.png', '.heic')):
                            writer.writerow([image_name, rank, suit])

    def load_images(self, target_size=(128, 128)):
        """Loads and resizes images, returning them as numpy arrays."""
        images, labels = [], []

        # Get total number of images for progress tracking
        total_images = sum([len(files) for _, _, files in os.walk(self.dataset_path)])

        # Progress bar using tqdm
        with tqdm(total=total_images, desc="Loading images", unit="image") as pbar:
            for folder_name in os.listdir(self.dataset_path):
                folder_path = os.path.join(self.dataset_path, folder_name)
                if os.path.isdir(folder_path):
                    rank, suit = folder_name.split('_')
                    for img_name in os.listdir(folder_path):
                        # Only attempt to open files with supported image extensions
                        if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.heic')):
                            img_path = os.path.join(folder_path, img_name)
                            try:
                                img = Image.open(img_path).resize(target_size)
                                images.append(np.array(img) / 255.0)
                                labels.append(f"{rank}_{suit}")
                            except Exception as e:
                                print(f"Could not open image {img_path}: {e}")
                            finally:
                                pbar.update(1)  # Update progress bar

        return images, labels

# Initialize DataLoader
data_loader = DataLoader(r"C:\Users\jobri\OneDrive - Drexel University\MEM679\cropped_cards_data")
images, labels = data_loader.load_images()



In [ ]:
# Visualization
import matplotlib.pyplot as plt

class Visualizer:
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def display_sample_images(self, num_samples=5):
        """Displays a random selection of images from the dataset."""
        sample_indices = np.random.choice(len(self.images), num_samples, replace=False)
        plt.figure(figsize=(10, 5))
        for i, idx in enumerate(sample_indices, 1):
            plt.subplot(1, num_samples, i)
            plt.imshow(self.images[idx])
            plt.title(self.labels[idx])
            plt.axis("off")
        plt.show()

# Initialize Visualizer and display sample images
visualizer = Visualizer(images, labels)
visualizer.display_sample_images(num_samples=5)


In [ ]:
import panel as pn

# Load the Panel extension
pn.extension()

# Interactive Visualization using Panel
def display_image_selector(images, labels):
    """Panel interface to select and view individual images."""
    def view_image(idx):
        plt.imshow(images[idx])
        plt.title(labels[idx])
        plt.axis("off")
        plt.show()
    
    return pn.interact(view_image, idx=(0, len(images) - 1))

# Display interactive selector
display_image_selector(images, labels)
